# Download train and test data from here
https://inclass.kaggle.com/c/gene-expression-prediction/

In [11]:
#coding: utf-8

In [12]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

In [13]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import train_test_split

In [14]:
from sklearn.decomposition import PCA, KernelPCA
from xgboost import XGBClassifier

# Read data
here the reader was prepared by @author: lingyu, hehu

In [53]:
data_path = "/Users/kl0niq/Documents/Skoltech/Machine_learning/kaggle2" # This folder holds the csv files

# load csv files. We use np.loadtxt. Delimiter is ","
# and the text-only header row will be skipped.

print("Loading data...")
x_train = np.loadtxt(data_path + os.sep + "x_train.csv", 
                     delimiter = ",", skiprows = 1)
x_test  = np.loadtxt(data_path + os.sep + "x_test.csv", 
                     delimiter = ",", skiprows = 1)    
y_train = np.loadtxt(data_path + os.sep + "y_train.csv", 
                     delimiter = ",", skiprows = 1)

print("All files have been loaded.")

Loading data...
All files have been loaded.


In [54]:
# remove the first column(Id)
x_train = x_train[:,1:] 
x_test  = x_test[:,1:]   
y_train = y_train[:,1:] 

In [56]:
# Every 100 rows correspond to one gene.
# Extract all 100-row-blocks into a list using np.split.
num_genes_train = x_train.shape[0] / 100
num_genes_test  = x_test.shape[0] / 100

print("Train / test data has %d / %d genes." % \
      (num_genes_train, num_genes_test))

x_train = np.split(x_train, num_genes_train)
x_test  = np.split(x_test, num_genes_test)

Train / test data has 15485 / 3871 genes.


In [57]:
# Reshape by raveling each 100x5 array into a 500-length vector
x_train = [g.ravel() for g in x_train]
x_test  = [g.ravel() for g in x_test]

In [58]:
# convert data from list to array
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test  = np.array(x_test)
y_train = np.ravel(y_train)

In [59]:
# Now x_train should be 15485 x 500 and x_test 3871 x 500.
# y_train is 15485-long vector.

print("x_train shape is %s" % str(x_train.shape))    
print("y_train shape is %s" % str(y_train.shape))
print("x_test shape is %s" % str(x_test.shape))

print('Data preprocessing done...')

x_train shape is (15485, 500)
y_train shape is (15485,)
x_test shape is (3871, 500)
Data preprocessing done...


In [60]:
x_train_train, x_train_test, y_train_train, y_train_test = train_test_split(x_train, y_train, train_size = 0.7)

# Play with the learned methods
Using k-fold cross-validation compare the performance (train and prediction time, roc_auc_score, accuracy) of these ones:
* LDA
* QDA
* Logistic Regression (with different regularizations)
* xgboost

In [67]:
def test_methods(X_train, X_test, y_train, y_test):
    methods = [XGBClassifier(max_depth=10, n_estimators=500)]
#     LDA(), QDA(), LogisticRegression(C=0.01, warm_start=True, penalty='l1'), 
    method_names = ['XGBOOST']
#     'LDA', 'QDA', 'LR', 
    accuracies = []
    roc_aucs = []
    for i in range(len(methods)):
        methods[i].fit(X_train, y_train)
        y_pred = methods[i].predict(X_test)
        y_prob = methods[i].predict_proba(X_test)[:, 1]
        accuracies.append(metrics.accuracy_score(y_test, y_pred))
        roc_aucs.append(metrics.roc_auc_score(y_test, y_prob))
    print('accuracy:')
    for i in range(len(methods)):
        print(method_names[i], accuracies[i])
    print('\nroc_auc:')
    for i in range(len(methods)):
        print(method_names[i], roc_aucs[i])

In [68]:
test_methods(x_train_train, x_train_test, y_train_train, y_train_test)

accuracy:
XGBOOST 0.854068015497

roc_auc:
XGBOOST 0.920419270846


In [69]:
method = XGBClassifier(max_depth=10, n_estimators=500)
method.fit(x_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

# Prepare a submission file for the best method
## <font color='red'>There are only 2 submissions allowed per day, use them wisely</font>
For example, try to submit a tuned Logistic Regression and xgboost Classifier

In [71]:
y_test_pred_proba = method.predict_proba(x_test)[:, 1]
with open('submission.csv', 'w') as out:
    out.write('GeneId,Prediction\n')
    for i in range(len(y_test_pred_proba)):
        out.write(str(i + 1) + ',' + ('%.5f' % y_test_pred_proba[i]) + '\n')